# scRFE Starter CodeV3

In [51]:
# 2020-05-06
# getLabels Function, adding makeOneClassForest Function 

In [1]:
# Imports 
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import read_h5ad
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

In [11]:
# read in data
adata = read_h5ad('/Users/madelinepark/Downloads/Kidney_facs.h5ad')
# adata = read_h5ad('/Users/madelinepark/Downloads/Heart_droplet.h5ad')

In [12]:
# goal: get labels on a per class basis that will go into randomForest function for y
# THE ONE I HAVE BEEN USING THAT WORKS 
def getLabels (dataMatrix, classOfInterest): 
    labelsDict = {}
    for label in np.unique(adata.obs[classOfInterest]):
        lists = []
        for obs in adata.obs[classOfInterest]:
            if obs == label: 
                lists.append('A')
            else:
                lists.append('B')
            labelsDict[obs] = lists
    return labelsDict

In [14]:
# call getLabels
labelsDict = getLabels(dataMatrix = adata, classOfInterest = 'age')
# labelsDict

In [16]:
# getLabels Unit Test Cases 
# # data with one class, two classes, and two+ classes 
# # droplet vs facs? maybe this doesn't matter 
kidneyFacs_getLabels_unitTest = getLabels(dataMatrix = '/Users/madelinepark/Downloads/Kidney_facs.h5ad', classOfInterest = 'age')
heartDroplet_getLabels_unitTest = getLabels(dataMatrix = '/Users/madelinepark/Downloads/Heart_droplet.h5ad', classOfInterest = 'age')


### Figure out why it works below when it's not in a loop but breaks when it's a loop and why it is printing same results

In [304]:
dataMatrix = adata
classOfInterest = 'age'
labelsDict = getLabels(dataMatrix, classOfInterest) 
results_df = pd.DataFrame() 
for label in list(labelsDict.keys()):
    print(label)
    feat_labels = dataMatrix.var_names #this is equivalent of the genes
    X = dataMatrix.X
    y1 = np.array([labelsDict[label]])#series of strings, converted from list
    y = y1.transpose() #tryin
    print(type(X))
    print((y.shape))
    clf = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1, oob_score=True)
    selector = RFECV(clf, step=0.2, cv=3) # step = % rounded down at each iteration
    clf.fit(adata.X, y) #confused if this y is labels or the genes because feat_labels is genes
    selector.fit(X, y)
#     working from here up 
    feature_selected = feat_labels[selector.support_] 

    print('result writing')
    column_headings = []
    column_headings.append(c)
    column_headings.append(c + '_gini')
    resaux = pd.DataFrame(columns=column_headings)
    resaux[c] = feature_selected
    resaux[c + '_gini'] = (selector.estimator_.feature_importances_)

    print(feature_selected)
    print (selector.estimator_.feature_importances_)
    results_df = pd.concat([results_df,resaux],axis=1)

18m
<class 'scipy.sparse.csr.csr_matrix'>
(7916, 1)


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['Xkr4', 'Rp1', 'Sox17', 'Mrpl15', 'Lypla1', 'Tcea1', 'Rgs20', 'Atp6v1h',
       'Oprk1', 'Npbwr1',
       ...
       'Uty', 'Ddx3y', 'Usp9y', 'Rbmy1a1', 'LOC100039753', 'LOC434960',
       'LOC380994', 'LOC100041346', 'Sly', 'Erdr1'],
      dtype='object', name='index', length=19860)
[0. 0. 0. ... 0. 0. 0.]
21m
<class 'scipy.sparse.csr.csr_matrix'>
(7916, 1)


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

...

/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['Xkr4', 'Rp1', 'Sox17', 'Mrpl15', 'Lypla1', 'Tcea1', 'Rgs20', 'Atp6v1h',
       'Oprk1', 'Npbwr1',
       ...
       'Uty', 'Ddx3y', 'Usp9y', 'Rbmy1a1', 'LOC100039753', 'LOC434960',
       'LOC380994', 'LOC100041346', 'Sly', 'Erdr1'],
      dtype='object', name='index', length=19860)
[0. 0. 0. ... 0. 0. 0.]
3m
<class 'scipy.sparse.csr.csr_matrix'>
(7916, 1)


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['Xkr4', 'Rp1', 'Sox17', 'Mrpl15', 'Lypla1', 'Tcea1', 'Rgs20', 'Atp6v1h',
       'Oprk1', 'Npbwr1',
       ...
       'Uty', 'Ddx3y', 'Usp9y', 'Rbmy1a1', 'LOC100039753', 'LOC434960',
       'LOC380994', 'LOC100041346', 'Sly', 'Erdr1'],
      dtype='object', name='index', length=19860)
[0. 0. 0. ... 0. 0. 0.]


In [306]:
# results_df

In [8]:
# is it printing same results bc of how I am looping? like it's getting over ridden?
def makeOneClassForest(dataMatrix, classOfInterest, step=0.2, cv=3,  n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True, min_samples_split=2, min_samples_leaf=1):
#     results_df = pd.DataFrame() 
    results_df = pd.DataFrame() 
    labelsDict = getLabels(dataMatrix, classOfInterest) 
    print(labelsDict.keys())
    for label in list(labelsDict.keys())[0:3]:
        print(label)
        feat_labels = dataMatrix.var_names #this is equivalent of the genes
        X = dataMatrix.X
        y1 = np.array([labelsDict[label]])#series of strings, converted from list
        y = y1.transpose() 
        print(type(X))
        print((y.shape))
        clf = RandomForestClassifier(n_estimators, random_state, n_jobs, oob_score, min_samples_split, min_samples_leaf)
        selector = RFECV(clf, step, cv) #when I hardcode args for this and clf, it does not error
        
        clf.fit(X, y)
        selector.fit(X, y)
        feature_selected = feat_labels[selector.support_] 
        
        print('result writing')
        column_headings = []
        column_headings.append(c)
        column_headings.append(c + '_gini')
        resaux = pd.DataFrame(columns=column_headings)
        resaux[c] = feature_selected
        resaux[c + '_gini'] = (selector.estimator_.feature_importances_)

        print(feature_selected)
        print (selector.estimator_.feature_importances_)
        results_df = pd.concat([results_df,resaux],axis=1)
    results_df 

In [9]:
makeOneClassForest(dataMatrix = adata, classOfInterest = 'age', step=0.2, cv=3,  n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True, min_samples_split=2, min_samples_leaf=1)

dict_keys(['18m', '21m', '24m', '30m', '1m', '3m'])
18m
<class 'scipy.sparse.csr.csr_matrix'>
(7916, 1)


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer True